# Unmapped Metabase

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Catalog/Unmapped/data'
STORE = 'vinoteca'

## Import

In [2]:
import sys
sys.path.append('/Users/efraflores/Desktop/hub/cornershop/venv/lib/python3.9/site-packages')

In [3]:
import os
import time
import numpy as np
import pandas as pd
from IPython.lib.display import Audio

prod = pd.read_csv(os.path.join(BASE_DIR,f'prod_{STORE}.csv')).drop_duplicates()
supply = pd.read_csv(os.path.join(BASE_DIR,f'supply_{STORE}.csv')).drop_duplicates()

display(prod.sample())
display(supply.sample())

,store_id,catalog_product_id,barcodes,branch,productbranch_id,price
16462,97,2255860,7501005600811,904,60561833,463.0


,sku_source,supply_product_id,catalog_product_id,name,barcode,branch,price_mode,stock,min_last_visit
804,mx-vinoteca-csv,14145012,1155877.0,Tenedor grande biodegradable,7.500463e+12,903,31.45,21,2021-07-01T01:25:29.133928Z


## Functions

### Timing and tone

In [4]:
start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

### Batches

In [5]:
def batches(base_dir,store,data,size=10000):
    df = data.copy()
    n = 0
    for i in range(int(np.ceil(len(df)/size))):
        batch = df.iloc[n:n+size-1,:]
        batch.to_excel(os.path.join(base_dir,f'bulk_{store}_{str(i+1).zfill(2)}.xlsx'),index=False)
        n += size
    print(f'The file with {len(df)} rows has been split up into {i+1} batches of {size} rows each')

### Summary

In [25]:
def summary():
    print(f"Para la tienda {STORE.title()}, los productbranch se distribuyen:\
    \n-Catálogo\t{len(prod)}\
    \n-Supply\t\t{len(supply)}\
    \n-Sin mapear\t{len(unmapped)+len(brands)}\t({'{:.1%}'.format((len(unmapped)+len(brands))/len(full))})\
    \n -Brands\t{len(branch_brands)}\
    \n -Sin órdenes\t{len(df[df['order_products']==0])}\
    \n -Buen perform\t{len(good_perf)}\t(FR {'{:.1%}'.format(good_perf['fr'].mean())})\
    \n -Mal perform\t{len(bad_fr)}\t(FR {'{:.1%}'.format(bad_fr['fr'].mean())})\
    \n -Markup +-3%\t{len(mk)}\t(MK {int(mk['mk_tot'].sum())}, {'{:.1%}'.format(mk['mk_tot'].sum()/mk['sales'].sum())})\
    \n\n-Por desactivar\t{len(bad_perf)}\t({'{:.1%}'.format(len(bad_perf)/len(prod))})\
    \n -Productos \t{len(bad_perf.groupby('catalog_product_id')['store_id'].count())}\
    \n -Orders_prod\t{int(bad_perf['order_products'].sum())}\t({'{:.1%}'.format(bad_perf['order_products'].sum()/total['order_products'].sum())})\
    \n -Ventas\t$ {int(bad_perf['sales'].sum())}\t({'{:.1%}'.format(bad_perf['sales'].sum()/total['sales'].sum())})")

## Unmapped from prod

In [7]:
full = prod.merge(supply,how='left',on=['branch','catalog_product_id'])
full.isnull().mean()

store_id              0.000000
catalog_product_id    0.000000
barcodes              0.000000
branch                0.000000
productbranch_id      0.000000
price                 0.000000
sku_source            0.016657
supply_product_id     0.016657
name                  0.016657
barcode               0.016657
price_mode            0.016657
stock                 0.016657
min_last_visit        0.016657
dtype: float64

In [8]:
unmapped = full[full['supply_product_id'].isnull()].dropna(axis=1)
#unmapped.to_csv(os.path.join(BASE_DIR,f'check_brands_{STORE}.csv'),index=False)

## Brands

Check with @Alan Sánchez Márquez if any product_id was created by Brands

In [9]:
brands = pd.read_csv(os.path.join(BASE_DIR,f'brands.csv'))
brands = brands[brands['Integración']=='Sin mapear'].copy()
brands.sample(3)

,Product ID,Origen,Barcode (EAN),Nombre del producto,Marca,Package (Gramaje),Fecha de creación,Días de vida,Precio,Store,Integración,NOTAS,Gsheet KAM,Gsheet terminado,PBI,PBI2,Status Final
2174,568184,Editado,630509288472,NaN,NaN,NaN,17/12/2020,-194.0,$169.00,Soriana,Sin mapear,NaN,NaN,NaN,NaN,NaN,Desactivado.
4799,3682930,Creado,7503029963576,Vino blanco,Rivero González,750 ml,19/4/2021,-71.0,$333.00,La Comer,Sin mapear,NaN,https://docs.google.com/spreadsheets/d/1TWZHv1...,https://docs.google.com/spreadsheets/d/1UlxdBh...,NaN,NaN,Desactivado.
6551,3594561,Editado,7613287066893,Supreme pro 1 fórmula para lactantes,Nan,400 g,14/6/2021,-15.0,$189.00,Sumesa,Sin mapear,NaN,https://docs.google.com/spreadsheets/d/1lPcumt...,https://docs.google.com/spreadsheets/d/1xbwwiD...,NaN,NaN,NaN


In [10]:
import re
brands['inactive'] = brands['Status Final'].apply(lambda x:re.search('desac',str(x),flags=re.I)!=None)*1
brands.rename(columns={'Product ID':'catalog_product_id'},inplace=True)
brands = brands[(brands['Status Final'].notnull())&(brands['inactive']==0)]
brands[['catalog_product_id','inactive']].sample(4)

,catalog_product_id,inactive
2062,1137576,0
6770,3901599,0
2113,1137595,0
733,2721571,0


In [11]:
brands[brands['inactive']==0]['Status Final'].value_counts(0,dropna=False)

Activo (mapeado).    39
Reactivado.           2
Buen FR.              1
Store.                1
Name: Status Final, dtype: int64

In [12]:
branch_brands = unmapped[unmapped['catalog_product_id'].isin(brands['catalog_product_id'])]
unmapped = unmapped[~unmapped['catalog_product_id'].isin(brands['catalog_product_id'])].copy()
#','.join(set(unmapped['catalog_product_id'].dropna().astype(int).astype(str)))

## Orders from unmapped

In [14]:
orders = pd.read_csv(os.path.join(BASE_DIR,f'orders_{STORE}.csv'))
orders.head()

,level,date,catalog_product_id,branch,productbranch_id,min_branch_price,avg_branch_price,max_branch_price,min_shopper_price,avg_shopper_price,max_shopper_price,requested,found,order_products,sales
0,TOTAL,2021-06-09,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,126.0,103.0,71,46901.23
1,TOTAL,2021-06-10,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,145.0,131.0,87,44755.47
2,TOTAL,2021-06-11,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,265.0,227.0,146,82168.96
3,TOTAL,2021-06-12,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,362.0,317.0,171,85022.29
4,TOTAL,2021-06-13,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,20.0,20,7521.45


In [15]:
total = orders[orders['level']=='TOTAL'].set_index('date').iloc[:,-4:].copy()
total['fr'] = total['found']/total['requested']
total

,requested,found,order_products,sales,fr
date,,,,,
2021-06-09,126.0,103.0,71,46901.23,0.817460
2021-06-10,145.0,131.0,87,44755.47,0.903448
2021-06-11,265.0,227.0,146,82168.96,0.856604
2021-06-12,362.0,317.0,171,85022.29,0.875691
2021-06-13,23.0,20.0,20,7521.45,0.869565
2021-06-14,171.0,165.0,61,36169.52,0.964912
2021-06-15,162.0,134.0,83,51848.22,0.827160
2021-06-16,163.0,143.0,98,67797.61,0.877301
2021-06-17,314.0,228.0,160,95670.40,0.726115


### Total

In [16]:
df = unmapped.set_index('catalog_product_id').merge(orders.iloc[:,1:],on='productbranch_id',how='left').fillna(0)
df = df[~df['catalog_product_id'].isin(brands['catalog_product_id'])].copy()
df = df.sort_values('order_products',0,0).reset_index(drop=True)
df['fr'] = (df['found']/(df['requested']+1e-10)).apply(lambda x: min(1,x))
print(df['fr'].describe())
df.head()

count    455.000000
mean       0.804029
std        0.395105
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: fr, dtype: float64


,store_id,barcodes,branch_x,productbranch_id,price,date,catalog_product_id,branch_y,min_branch_price,avg_branch_price,max_branch_price,min_shopper_price,avg_shopper_price,max_shopper_price,requested,found,order_products,sales,fr
0,97,5011013100156,893,45768796,317.00,2021-06-19,1993289.0,893.0,317.00,317.00,317.00,317.00,326.333333,345.00,3.0,3.0,3.0,951.00,1.0
1,97,5011013100132,893,45797034,318.90,2021-06-26,1897939.0,893.0,318.90,318.90,318.90,318.90,318.900000,318.90,4.0,4.0,3.0,1275.60,1.0
2,97,5011013100132,893,45797034,318.90,2021-06-24,1897939.0,893.0,318.90,318.90,318.90,318.90,331.950000,345.00,2.0,2.0,2.0,637.80,1.0
3,97,7503002087206,893,8538019,398.97,2021-06-19,389004.0,893.0,398.97,398.97,398.97,398.97,398.970000,398.97,2.0,2.0,2.0,797.94,1.0
4,97,5000267117201,910,50875187,946.90,2021-06-18,2065172.0,910.0,946.90,946.90,946.90,722.25,834.575000,946.90,5.0,5.0,2.0,4734.50,1.0


### Bad performance

BULK TOOL

<https://cornershopapp.com/catalog/dashboard/bulk_update/product_branches/>

In [17]:
df['catalog_product_id'] = df['catalog_product_id'].astype(int).astype(str)
df = df.groupby(['store_id','productbranch_id',
                 'catalog_product_id']).agg({'price':'median','requested':sum,
                                             'found':sum,'sales':sum,
                                             'order_products':sum}).reset_index()

In [18]:
df['fr'] = (df['found'] / (df['requested']+1e-10)).apply(lambda x: min(x,1))
df['avg_price'] = df['sales'] / df['found']
df['mk_un'] = df['avg_price'] - df['price']
df['mk_p'] = df['mk_un'] / df['avg_price']
df['mk_tot'] = df['sales'] - df['price']*df['found']
bad_fr = df[(df['fr']<=total['fr'][0]*.9)
            #&(df['orders']>0)
           ].copy()
mk = df[(df['mk_p']<-.03)|(df['mk_p']>.03)].copy()
bad_perf = bad_fr.append(mk)
print(bad_perf['fr'].describe())
bad_perf.head()

count    72.000000
mean      0.035995
std       0.160623
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000
Name: fr, dtype: float64


,store_id,productbranch_id,catalog_product_id,price,requested,found,sales,order_products,fr,avg_price,mk_un,mk_p,mk_tot
0,97,8270813,0,368.05,0.0,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,0.0
4,97,8273497,0,449.00,0.0,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,0.0
5,97,8274573,0,239.00,0.0,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,0.0
10,97,8276019,384313,255.00,3.0,2.0,510.0,3.0,0.666667,255.0,0.0,0.0,0.0
12,97,8276104,0,475.00,0.0,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,0.0


### Good performance

Ask the KAM to talk to the partner in order to activate that product

In [19]:
good_perf = df[(~df['productbranch_id'].isin(bad_perf['productbranch_id']))&
               (df['order_products']>0)].copy()
good_perf.to_csv(os.path.join(BASE_DIR,f'good_perf_{STORE}.csv'),index=False)
print(good_perf['fr'].describe())
good_perf.head()

count    196.000000
mean       0.997874
std        0.021411
min        0.750000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: fr, dtype: float64


,store_id,productbranch_id,catalog_product_id,price,requested,found,sales,order_products,fr,avg_price,mk_un,mk_p,mk_tot
1,97,8270821,383784,349.0,5.0,5.0,1745.0,1.0,1.0,349.0,0.0,0.0,0.0
2,97,8271382,383678,249.0,1.0,1.0,249.0,1.0,1.0,249.0,0.0,0.0,0.0
3,97,8273287,384249,266.0,6.0,6.0,1596.0,2.0,1.0,266.0,0.0,0.0,0.0
6,97,8275382,383676,1229.8,1.0,1.0,1229.8,1.0,1.0,1229.8,0.0,0.0,0.0
7,97,8275426,383720,599.0,1.0,1.0,599.0,1.0,1.0,599.0,0.0,0.0,0.0


## Bulk

In [20]:
bad_perf.to_csv(os.path.join(BASE_DIR,f'bad_performance_{STORE}.csv'),index=False)

In [21]:
export = bad_perf[['store_id','productbranch_id']].copy().drop_duplicates()
export.rename(columns={'productbranch_id':'product_branch_id'},inplace=True)
export['price'] = ''
export['availability'] = 'UNAVAILABLE'
export['schedule'] = ''
export.head()

,store_id,product_branch_id,price,availability,schedule
0,97,8270813,,UNAVAILABLE,
4,97,8273497,,UNAVAILABLE,
5,97,8274573,,UNAVAILABLE,
10,97,8276019,,UNAVAILABLE,
12,97,8276104,,UNAVAILABLE,


## Summary

In [22]:
batches(BASE_DIR,STORE,export)

The file with 72 rows has been split up into 1 batches of 10000 rows each


In [23]:
time_exp(time.time()-start)
tono()

2 minutos con 6.01 segundos


In [26]:
summary()

Para la tienda Vinoteca, los productbranch se distribuyen:    
-Catálogo	16502    
-Supply		20718    
-Sin mapear	367	(1.9%)    
 -Brands	0    
 -Sin órdenes	68    
 -Buen perform	196	(FR 99.8%)    
 -Mal perform	71	(FR 2.2%)    
 -Markup +-3%	1	(MK -223, -20.0%)    

-Por desactivar	72	(0.4%)    
 -Productos 	5    
 -Orders_prod	20	(0.6%)    
 -Ventas	$ 18282	(1.0%)
